In [1]:
import numpy as np
import numpy.random as random

In [3]:
import importlib

In [9]:
import LinearBoard

In [13]:
importlib.reload(LinearBoard);

In [16]:
b = LinearBoard.LinearBoard(3)
b.board[0]=1
b.board[1]=2
b.board[2]=1
b.board[ b.ij_to_ind(1,0)] = 2
b.board[ b.ij_to_ind(2,0)] = 2
b.board[ b.ij_to_ind(1,1)] = 2
b.board[ b.ij_to_ind(2,1)] = 1
b.board[ b.ij_to_ind(1,2)] = 1
b.board[ b.ij_to_ind(2,2)] = 1
b.printBoard()

    0 1 2 
0  |X 0 X |
1  |0 0 X |
2  |0 X X |


In [17]:
b.getWinner()

1

In [104]:
b.getMoves()

[4, 5, 6, 7, 8]

In [109]:
def getWinner(board, masks):
    for m in masks:
        w = np.unique(board[m])
        if len(w)==1 and w[0]>0:
            return w[0]
    if np.sum(board==0)==0:
        return 0
    return -1

In [110]:
getWinner(b.board, b.masks)

-1

In [88]:
np.unique( b.board[ b.masks[0] ] )

array([1, 2])

## Backup 

In [3]:
import board

In [73]:
importlib.reload(board);

In [74]:
B = board.Board(n=3)
B.movesToBoard([ (1,[0,0]), (1,[0,1]), (2,[0,2])])
B.printBoard()

    0 1 2 
0  |X X 0 |
1  |      |
2  |      |
Winner: -1


In [75]:
B2 = B.copy()

In [70]:
def copy(b):
    newB = board.Board()
    newB.board = b.board.copy()
    newB.masks = b.masks.copy()
    return newB

In [76]:
B2 = B.copy()
B2.board[1][0]=1
B2.printBoard()


    0 1 2 
0  |X X 0 |
1  |X     |
2  |      |
Winner: -1


In [77]:
B.printBoard()

    0 1 2 
0  |X X 0 |
1  |      |
2  |      |
Winner: -1


In [78]:
def simulateGame(B, p1=None, p2=None, rnd=0, playerToMove_ = 1, max_moves = 10000):
    history = B.boardToMoves()
    board = B.board
    masks = B.masks
    n_moves = 0
    playerToMove = playerToMove_
    while B.getWinner() == -1 and n_moves<max_moves:
        n_moves = n_moves+1
        # Chose a move (random or use a player model if provided)
        move = None
        moves = B.getMoves()
        move = moves[random.randint(0, len(moves) )]
       
        # Make the move
        B.board[move[0]][move[1]] = playerToMove
        
        # Add the move to the history
        history.append((playerToMove, move))
        
        # Switch the active player
        playerToMove = 1 if playerToMove == 2 else 2
        
    return history

In [230]:
B = board.Board(5)
simulateGame(B, max_moves=1);
B.printBoard()

    0 1 2 3 4 
0  |          |
1  |X         |
2  |          |
3  |          |
4  |          |
Winner: -1


In [236]:
moves = B.getMoves()
probe_MC = np.zeros( len(moves))
for iMove in range(len(moves)):
    m = moves[iMove]
    probe_results = np.empty([], dtype = 'int')
    for _ in range(100):
        B2 = B.copy()
        B2.board[m[0]][m[1]] = 2
        simulateGame(B2, playerToMove_=1)
        probe_results = np.append(probe_results, B2.getWinner())
    probe_MC[iMove] = np.mean( probe_results == 2)

In [235]:
best_move = moves[np.argmax(probe_MC)]
best_move

[0, 4]

In [229]:
B.board[best_move[0]][best_move[1]] = 2;
B.printBoard()

    0 1 2 3 4 
0  |    0     |
1  |      X   |
2  |    0     |
3  |          |
4  |          |
Winner: -1
